In [1]:
!pip uninstall numba -y
!pip install -U numba

Found existing installation: numba 0.54.1
Uninstalling numba-0.54.1:
  Successfully uninstalled numba-0.54.1
  Using cached numba-0.54.1-cp37-cp37m-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.3 MB)


In [2]:
%matplotlib inline

import numpy as np
import scipy.linalg
import scipy.sparse.linalg
import matplotlib.pyplot as plt
from numba import jit

import numba
print(numba.__version__)

0.54.1


In [3]:
@jit(nopython=True)
def get_combination(NOS,NOD):
    combination = np.zeros((NOS,NOD),dtype=np.int64)
    for i in range(NOS):
        for j in range(NOD):
            combination[i,j] = f_combination(i+1,j+1)
    return combination

In [4]:
@jit(nopython=True)
def f_combination(n,k):
    if n<k:
        return 0
    nCk = 1
    for i in range(1,k+1):
        nCk = nCk * (n-k+i)
        nCk = nCk//i
    return nCk

In [5]:
@jit(nopython=True)
def insertion_sort(a,NOD):
    for i in range(2,NOD+1):
        j = i - 1
        temp = a[i-1]
        while a[j-1] > temp:
            a[j] = a[j-1]
            j = j - 1
            if j==0:
                break
        a[j] = temp
    return 0

In [6]:
@jit(nopython=True)
def inv_list(ni,NOD,combination):
    val_inv_list = ni[0]
    for i in range(2,NOD+1):
        val_inv_list = val_inv_list + combination[ni[i-1]-2,i-1]
    return val_inv_list

In [7]:
@jit(nopython=True)
def qsort_w_order(a,o,first,last):
    x = a[(first+last)//2-1]
    i = first
    j = last
    while True:
        while a[i-1] < x:
            i = i + 1
        while x < a[j-1]:
            j = j - 1
        if i >= j:
            break
        t8 = a[i-1];  a[i-1] = a[j-1];  a[j-1] = t8
        t  = o[i-1];  o[i-1] = o[j-1];  o[j-1] = t
        i = i + 1
        j = j - 1
    if first < i - 1:
        qsort_w_order(a,o,first,i-1)
    if j + 1 < last:
        qsort_w_order(a,o,j+1,last)
    return 0

In [8]:
## output "ni" is returned
@jit(nopython=True)
def list_fly(t,NOD,NOS,combination):
    ni = np.zeros(NOD,dtype=np.int64)
    s = t
    j = NOS - 1
    for i in range(NOD,1,-1):
        b, j0 = binary_search(s,combination[:,i-1],i,j)
        j = j0 - 1
        ni[i-1] = j0
        s = s - combination[j-1,i-1]
    ni[0] = s
    return ni

In [9]:
## output "ni" is in arguments
@jit(nopython=True)
def list_fly_2(t,NOD,NOS,combination,ni):
    ni[:] = 0
    s = t
    j = NOS - 1
    for i in range(NOD,1,-1):
        b, j0 = binary_search(s,combination[:,i-1],i,j)
        j = j0 - 1
        ni[i-1] = j0
        s = s - combination[j-1,i-1]
    ni[0] = s
    return 0

In [10]:
@jit(nopython=True)
def binary_search(s,list_s,ls,le):
    bmin = ls; bmax = le
    while True:
        b = bmin + (bmax-bmin)//2
        if s < list_s[b-1]:
            bmax = b - 1
        elif list_s[b-1] < s:
            bmin = b + 1
        else:
            bmin = b
            return b, bmin
        if bmin > bmax:
            b = -1
            return b, bmin
    return b, bmin

In [11]:
@jit(nopython=True)
def list_to_state_no_duplication(st_list,NOS):
    string01 = ""
    for i in range(1,NOS+1):
        if i in st_list:
            string01 = string01 + "1" # down
        else:
            string01 = string01 + "0" # up
    return string01

In [12]:
#@jit(nopython=True)
def list_to_state(st_list,NOS):
    list01 = np.zeros(NOS,dtype=np.int64)
    for i in st_list:
        list01[i-1] += 1
    list01 = list01%2
    string01 = np.array2string(list01,separator="")[1:NOS+1]
    return string01

In [13]:
## output "nd" is returned
@jit(nopython=True)
def j_flip_ni(i,j,n,NOD):
    nd = np.ones(NOD,dtype=np.int64)
    kr = NOD
    for _kr in range(NOD,0,-1):
        if j < n[_kr-1]:
            kr = _kr
            continue
        elif j > n[_kr-1]:
            kr = _kr
            break
        else:
            nd[:] = 0
            kr = _kr
            break
    if nd[NOD-1] == 1: # S+_i S-_j
        kl = 1
        for _kl in range(1,kr+1):
            if i == n[_kl-1]:
                kl = _kl
                break
            kl = _kl+1
        nd[kl-1:kr-1] = n[kl:kr]
        nd[kr-1] = j
    else: # S-_i S+_j
        kl = 1
        for _kl in range(1,kr+1):
            if i < n[_kl-1]:
                kl = _kl
                break
            kl = _kl+1
        nd[kl-1] = i
        nd[kl:kr] = n[kl-1:kr-1]
    nd[0:kl-1] = n[0:kl-1]
    nd[kr:NOD] = n[kr:NOD]
    return nd

In [14]:
## output "nd" is in arguments
@jit(nopython=True)
def j_flip_ni_2(i,j,n,NOD,nd):
    nd[:] = 1
    kr = NOD
    for _kr in range(NOD,0,-1):
        if j < n[_kr-1]:
            kr = _kr
            continue
        elif j > n[_kr-1]:
            kr = _kr
            break
        else:
            nd[:] = 0
            kr = _kr
            break
    if nd[NOD-1] == 1: # S+_i S-_j
        kl = 1
        for _kl in range(1,kr+1):
            if i == n[_kl-1]:
                kl = _kl
                break
            kl = _kl+1
        nd[kl-1:kr-1] = n[kl:kr]
        nd[kr-1] = j
    else: # S-_i S+_j
        kl = 1
        for _kl in range(1,kr+1):
            if i < n[_kl-1]:
                kl = _kl
                break
            kl = _kl+1
        nd[kl-1] = i
        nd[kl:kr] = n[kl-1:kr-1]
    nd[0:kl-1] = n[0:kl-1]
    nd[kr:NOD] = n[kr:NOD]
    return 0

In [15]:
## output "Ham" is returned
@jit(nopython=True)
def make_full_hamiltonian(lv,combination,NOD,NOxxz,p_xxz,sJint,NOS):
    Ham = np.zeros((lv,lv),dtype=np.float64)
    for i in range(1,lv+1):
        st_list = list_fly(i,NOD,NOS,combination)
        for j in range(1,NOxxz+1):
            f1 = p_xxz[0,j-1] in st_list
            f2 = p_xxz[1,j-1] in st_list
            if f1^f2:
                Ham[i-1,i-1] = Ham[i-1,i-1] - sJint[j-1,1]
                ni = j_flip_ni(p_xxz[0,j-1],p_xxz[1,j-1],st_list,NOD)
                id = inv_list(ni,NOD,combination)
                Ham[i-1,id-1] = Ham[i-1,id-1] + sJint[j-1,0]
            else:
                Ham[i-1,i-1] = Ham[i-1,i-1] + sJint[j-1,1]
    return Ham

In [16]:
## output "Ham" is in arguments
@jit(nopython=True)
def make_full_hamiltonian_2(lv,Ham,combination,NOD,NOxxz,p_xxz,sJint,NOS):
    st_list = np.zeros(NOD,dtype=np.int64)
    ni = np.zeros(NOD,dtype=np.int64)
    for i in range(1,lv+1):
        list_fly_2(i,NOD,NOS,combination,st_list)
        for j in range(1,NOxxz+1):
            f1 = p_xxz[0,j-1] in st_list
            f2 = p_xxz[1,j-1] in st_list
            if f1^f2:
                Ham[i-1,i-1] = Ham[i-1,i-1] - sJint[j-1,1]
                j_flip_ni_2(p_xxz[0,j-1],p_xxz[1,j-1],st_list,NOD,ni)
                id = inv_list(ni,NOD,combination)
                Ham[i-1,id-1] = Ham[i-1,id-1] + sJint[j-1,0]
            else:
                Ham[i-1,i-1] = Ham[i-1,i-1] + sJint[j-1,1]
    return 0

In [17]:
@jit(nopython=True)
def make_parameters_1d(NOS,NOxxz):
    p_xxz = np.zeros((2,NOxxz),dtype=np.int64)
    Jint = np.zeros((NOxxz,2),dtype=np.float64) # Jint[NOxxz,0] --> Jint_x, Jint[NOxxz,1] --> Jint_z
    sJint = np.zeros((NOxxz,2),dtype=np.float64) # sJint[NOxxz,0] --> sJint_x, sJint[NOxxz,1] --> sJint_z
    for i in range(NOS):
        p_xxz[0,i] = i%NOS+1
        p_xxz[1,i] = (i+1)%NOS+1
        if p_xxz[0,i] > p_xxz[1,i]: # assume i<j for pair (i,j)
            tmp = p_xxz[0,i]
            p_xxz[0,i] = p_xxz[1,i]
            p_xxz[1,i] = tmp
    Jint[:,:] = 1.0
    sJint[:,0] = 0.5 * Jint[:,0]
    sJint[:,1] = 0.25 * Jint[:,1]
    return p_xxz, Jint, sJint

In [18]:
## memory allocation within get_vec
def ham_to_vec_wave_vector(lv,combination,NOD,NOxxz,p_xxz,sJint,NOS):
    @jit(nopython=True)
    def get_vec(v1): ## v0: new output, v1: old input
#        v0 = np.zeros(lv,dtype=np.complex128)
        v0 = np.zeros(lv,dtype=np.float64)
        for i in range(1,lv+1):
#            v0[i-1] = 0.0 + 0.0j
            v0[i-1] = 0.0
            st_list = list_fly(i,NOD,NOS,combination)
            for j in range(1,NOxxz+1):
                f1 = p_xxz[0,j-1] in st_list
                f2 = p_xxz[1,j-1] in st_list
                if f1^f2:
                    v0[i-1] = v0[i-1] - sJint[j-1,1] * v1[i-1]
                    ni = j_flip_ni(p_xxz[0,j-1],p_xxz[1,j-1],st_list,NOD)
                    id = inv_list(ni,NOD,combination)
                    v0[i-1] = v0[i-1] + sJint[j-1,0] * v1[id-1]
                else:
                    v0[i-1] = v0[i-1] + sJint[j-1,1] * v1[i-1]
        return v0
    return get_vec

In [19]:
## memory allocation outside get_vec
def ham_to_vec_wave_vector_2(lv,combination,NOD,NOxxz,p_xxz,sJint,NOS):
    @jit(nopython=True)
    def get_vec(v1,v0,st_list,ni): ## v0: new output, v1: old input
        for i in range(1,lv+1):
#            v0[i-1] = 0.0 + 0.0j
            v0[i-1] = 0.0
            list_fly_2(i,NOD,NOS,combination,st_list)
            for j in range(1,NOxxz+1):
                f1 = p_xxz[0,j-1] in st_list
                f2 = p_xxz[1,j-1] in st_list
                if f1^f2:
                    v0[i-1] = v0[i-1] - sJint[j-1,1] * v1[i-1]
                    j_flip_ni_2(p_xxz[0,j-1],p_xxz[1,j-1],st_list,NOD,ni)
                    id = inv_list(ni,NOD,combination)
                    v0[i-1] = v0[i-1] + sJint[j-1,0] * v1[id-1]
                else:
                    v0[i-1] = v0[i-1] + sJint[j-1,1] * v1[i-1]
        return v0
    return get_vec

In [20]:
def calculate_1d(NOS,NOD):
    #NOS = 4 # number of sites
    #NOD = 2 # number of down spins
    NOxxz = NOS # number of XXZ interaction
    combination = get_combination(NOS,NOD)
    THS = combination[NOS-1,NOD-1] # total Hilbert space
    print("# NOS,NOD")
    print(NOS,NOD)
    #print(combination)
    print("# total Hilbert space")
    print(THS)

    p_xxz, Jint, sJint = make_parameters_1d(NOS,NOxxz)
    #print(p_xxz)
    #print(Jint)
    #print(sJint)
    #print()

    get_vec = ham_to_vec_wave_vector_2(THS,combination,NOD,NOxxz,p_xxz,sJint,NOS)
    st_list = np.zeros(NOD,dtype=np.int64)
    ni = np.zeros(NOD,dtype=np.int64)
    #v0 = np.zeros(THS,dtype=np.complex128)
    v0 = np.zeros(THS,dtype=np.float64)
    Ham = scipy.sparse.linalg.LinearOperator((THS,THS),matvec=lambda v1: get_vec(v1,v0,st_list,ni))
    ene, vec = scipy.sparse.linalg.eigsh(Ham,which="SA",k=np.min([5,THS-1]))
    idx = ene.argsort()
    ene = ene[idx]
    vec = vec[:,idx]
    print("# energies")
    print(ene)
    print()
    #print("# vectors")
    #for i in range(len(ene)):
    #    print(i,vec[:,i])
    #print()

In [21]:
#for NOS in [4,8,16,32,64,128]: # number of sites
for NOS in [4,8,16,32,64]: # number of sites
    for NOD in [1,2,3]: # number of down spins
        calculate_1d(NOS,NOD)

# NOS,NOD
4 1
# total Hilbert space
4
# energies
[-1.00000000e+00 -8.92119745e-18  2.50310335e-17]

# NOS,NOD
4 2
# total Hilbert space
6
# energies
[-2.00000000e+00 -1.00000000e+00 -2.07835129e-17 -9.34747619e-19
  2.26339217e-17]

# NOS,NOD
4 3
# total Hilbert space
4
# energies
[-1.00000000e+00  1.16464203e-17  1.14729885e-16]

# NOS,NOD
8 1
# total Hilbert space
8
# energies
[8.85373054e-17 2.92893219e-01 2.92893219e-01 1.00000000e+00
 1.00000000e+00]

# NOS,NOD
8 2
# total Hilbert space
28
# energies
[-1.80193774 -1.2670351  -1.2670351  -1.14412281 -1.14412281]

# NOS,NOD
8 3
# total Hilbert space
56
# energies
[-3.12841906 -2.45873851 -2.45873851 -2.14514837 -2.14514837]

# NOS,NOD
16 1
# total Hilbert space
16
# energies
[2.         2.07612047 2.07612047 2.29289322 2.29289322]

# NOS,NOD
16 2
# total Hilbert space
120
# energies
[0.0437048  0.19283276 0.19283276 0.20823568 0.20823568]

# NOS,NOD
16 3
# total Hilbert space
560
# energies
[-1.81500162 -1.59933824 -1.59933824 -1.55